In [1]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('fer2013.csv')

# Separate features (pixels) and labels (emotions)
pixels = data['pixels'].tolist()
labels = data['emotion'].values

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('fer2013.csv')

# Preprocess the data
# Assuming the dataset has columns 'emotion' and 'pixels'
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))
X = np.stack(data['pixels'].values)
X = X.reshape(-1, 48, 48, 1)  # Reshape to 48x48 images with 1 channel
X = X / 255.0  # Normalize pixel values

y = to_categorical(data['emotion'], num_classes=7)  # Assuming 7 classes of emotions

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for emotion recognition
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.2f}')

# Save the model
model.save('emotion_model.h5')

/home/srtejaa/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-16 21:45:52.984695: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 30s 79ms/step - accuracy: 0.2403 - loss: 1.8252 - val_accuracy: 0.3969 - val_loss: 1.6189
Epoch 2/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 28s 77ms/step - accuracy: 0.3709 - loss: 1.6066 - val_accuracy: 0.4502 - val_loss: 1.4484
Epoch 3/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 29s 80ms/step - accuracy: 0.4253 - loss: 1.4868 - val_accuracy: 0.4993 - val_loss: 1.3394
Epoch 4/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 29s 80ms/step - accuracy: 0.4631 - loss: 1.3916 - val_accuracy: 0.5150 - val_loss: 1.2967
Epoch 5/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 28s 79ms/step - accuracy: 0.4783 - loss: 1.3554 - val_accuracy: 0.5240 - val_loss: 1.2623
Epoch 6/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 28s 79ms/step - accuracy: 0.4949 - loss: 1.3219 - val_accuracy: 0.5287 - val_loss: 1.2489
Epoch 7/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 28s 78ms/step - accuracy: 0.5065 - loss: 1.2859 - val_accuracy: 0.5505 - val_loss: 1.2017
Epoch 8/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 29s 80ms/step - accuracy: 0.5229 - loss: 1.2569 - 

Test accuracy: 0.58


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the data
data = pd.read_csv('fer2013.csv')

# Preprocess the data
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))
X = np.stack(data['pixels'].values)
X = X.reshape(-1, 48, 48, 1)  # Reshape to 48x48 images with 1 channel
X = X / 255.0  # Normalize pixel values

y = to_categorical(data['emotion'], num_classes=7)  # Assuming 7 classes of emotions

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Build the model
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for emotion recognition
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Callbacks for Learning Rate Scheduling, Early Stopping, and Model Checkpointing
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_emotion_model.keras', save_best_only=True)

# Train the model with data augmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                    epochs=50, 
                    validation_data=(X_test, y_test), 
                    callbacks=[lr_scheduler, early_stopping, model_checkpoint])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.2f}')

# Save the final model in the recommended Keras format
model.save('final_emotion_model.keras')


Epoch 1/50


/home/srtejaa/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srtejaa/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


898/898 ━━━━━━━━━━━━━━━━━━━━ 244s 269ms/step - accuracy: 0.2128 - loss: 13.3427 - val_accuracy: 0.3337 - val_loss: 9.8343 - learning_rate: 1.0000e-04
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 244s 271ms/step - accuracy: 0.2717 - loss: 8.8011 - val_accuracy: 0.3618 - val_loss: 6.1732 - learning_rate: 1.0000e-04
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 231s 257ms/step - accuracy: 0.3110 - loss: 5.5795 - val_accuracy: 0.3759 - val_loss: 3.9631 - learning_rate: 1.0000e-04
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 233s 259ms/step - accuracy: 0.3438 - loss: 3.6867 - val_accuracy: 0.4188 - val_loss: 2.8099 - learning_rate: 1.0000e-04
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 234s 260ms/step - accuracy: 0.3849 - loss: 2.7273 - val_accuracy: 0.4599 - val_loss: 2.2198 - learning_rate: 1.0000e-04
Epoch 6/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 234s 260ms/step - accuracy: 0.4202 - loss: 2.2431 - val_accuracy: 0.4666 - val_loss: 1.9766 - learning_rate: 1.0000e-04
Epoch 7/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 233s 259ms